### Conexión a las instancias de EC2 mediante el gestor de sesiones de AWS SSM

Problema
Tiene una instancia EC2 en una subred privada y necesita conectarse a la instancia sin usar SSH a través de Internet. 

Solución
Cree un rol de IAM, adjunte la política AmazonSSMManagedInstanceCore, cree un perfil de instancia EC2, adjunte el rol de IAM que creó al perfil de instancia, asocie el perfil de instancia EC2 a una instancia EC2 y, por último, ejecute el comando aws ssm start-session para conectarse a la instancia.
<br>
<br>
<img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781492092599/files/assets/awsc_0108.png" width="600">

In [1]:
import boto3
import json

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
iam = boto3.client('iam')

In [2]:
# Create a policy
policy_name = 'Cookbook-106-SSM-Role'
policy_document = {
  "Version": "2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": {
      "Service": "ec2.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
  }
  ]
}

ssm_role = iam.create_role(
    RoleName=policy_name,
    AssumeRolePolicyDocument=json.dumps(policy_document)
)

In [3]:
ssm_role_arn = ssm_role['Role']['Arn']

In [4]:
# Attach the policy to the role
iam.attach_role_policy(
    RoleName=policy_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
)

{'ResponseMetadata': {'RequestId': 'bc19038e-6d6e-4fa4-8b93-856b5183d93b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bc19038e-6d6e-4fa4-8b93-856b5183d93b',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Tue, 04 Oct 2022 13:48:12 GMT'},
  'RetryAttempts': 0}}

In [5]:
# Create an instance profile
instance_profile = iam.create_instance_profile(
    InstanceProfileName='Cookbook-106-SSM-Role'
)

In [6]:
# Add the role that you created to the instance profile:
iam.add_role_to_instance_profile(
    InstanceProfileName='Cookbook-106-SSM-Role',
    RoleName=policy_name
)

{'ResponseMetadata': {'RequestId': 'bf88e940-e56a-4960-8dea-73652e0d0903',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bf88e940-e56a-4960-8dea-73652e0d0903',
   'content-type': 'text/xml',
   'content-length': '228',
   'date': 'Tue, 04 Oct 2022 13:48:21 GMT'},
  'RetryAttempts': 0}}

El perfil de instancia de EC2 contiene un rol que usted crea. La asociación del perfil de instancia con una instancia le permite definir "quién soy", y el rol define "lo que se me permite hacer". Ambos son requeridos por IAM para permitir que una instancia EC2 se comunique con otros servicios de AWS utilizando el servicio IAM. Puede obtener una lista de perfiles de instancia en su cuenta ejecutando el comando `list-instance-profiles`.

In [7]:
# List instance profiles
iam.list_instance_profiles()

{'InstanceProfiles': [{'Path': '/',
   'InstanceProfileName': 'Cookbook-106-SSM-Role',
   'InstanceProfileId': 'AIPAYLG46Z2RRHT5O5QDC',
   'Arn': 'arn:aws:iam::573841526435:instance-profile/Cookbook-106-SSM-Role',
   'CreateDate': datetime.datetime(2022, 10, 4, 13, 48, 14, tzinfo=tzutc()),
   'Roles': [{'Path': '/',
     'RoleName': 'Cookbook-106-SSM-Role',
     'RoleId': 'AROAYLG46Z2RQDCI446LI',
     'Arn': 'arn:aws:iam::573841526435:role/Cookbook-106-SSM-Role',
     'CreateDate': datetime.datetime(2022, 10, 4, 13, 48, 8, tzinfo=tzutc()),
     'AssumeRolePolicyDocument': {'Version': '2012-10-17',
      'Statement': [{'Effect': 'Allow',
        'Principal': {'Service': 'ec2.amazonaws.com'},
        'Action': 'sts:AssumeRole'}]}}]}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '88954326-2e4a-465c-b3fc-dd11088cad87',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '88954326-2e4a-465c-b3fc-dd11088cad87',
   'content-type': 'text/xml',
   'content-length': '130

Consulte en SSM el último ID de AMI de Amazon Linux 2 disponible en su región y guárdelo como variable de entorno:

In [8]:
ssm = boto3.client('ssm')

ssm_response = ssm.get_parameter(
    Name='/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'
)

In [9]:
# Create a key pair
keypair = ec2.create_key_pair(KeyName='AWSCookbook-KeyPair-SSM')

In [10]:
# Launch an instance in one of your subnets that references the instance profile you created and also uses a Name tag that helps you identify the instance in the console:
instance = ec2.create_instances(
    ImageId=ssm_response['Parameter']['Value'],
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName='AWSCookbook-KeyPair-SSM',
    IamInstanceProfile={
        'Name': 'Cookbook-106-SSM-Role'
    },
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'Cookbook-106-SSM-Role'
                },
            ]
        },
    ]
)

Asegúrese de que su instancia EC2 se ha registrado en SSM. Utilice el siguiente comando para comprobar el estado. Este comando debería devolver el ID de la instancia:

In [14]:
instance_id = ssm.describe_instance_information()['InstanceInformationList'][0]['InstanceId']
instance_id

'i-01c3a553ddccacb10'

In [15]:
print("En la terminal ejecute el siguiente comando:")
print("   aws ssm start-session --target " + instance_id)

En la terminal ejecute el siguiente comando:
   aws ssm start-session --target i-01c3a553ddccacb10


Haz ping a un host en Internet para probar la conectividad a Internet:

In [ ]:
ping -c 4 homestarrunner.com